In [1]:

# Dashboard interativo com Bokeh: KPIs e múltiplos gráficos
from bokeh.layouts import column, row, gridplot
from bokeh.models import Panel, Tabs, Div, HoverTool, Select, CustomJS
from bokeh.palettes import Blues8, Greys8
from bokeh.transform import dodge

# Paleta de cores profissionais
azul_marinho = '#223A5E'
cinza = '#A9A9A9'
cinza_escuro = '#444444'

# Filtro de competência
select_competencia = Select(title="Filtrar por Competência:", 
                            value="Todas", 
                            options=["Todas"] + ordem_meses,
                            width=200)

# Filtragem inicial dos dados
data_filtrado = data.copy()

# 1. KPI: Total de bônus por ano
total_bonus = (data_filtrado['valor_bonus'].astype(float).sum() + 
               data_filtrado['rebate'].astype(float).sum() + 
               data_filtrado['trade'].astype(float).sum() + 
               data_filtrado['reembolso_ipva'].astype(float).sum())
kpi_total = Div(text=f"<h2 style='color:{azul_marinho}'>Total de Bônus: R$ {total_bonus:,.2f}</h2>")

# 2. Gráfico de barras: Bônus por competência (com HoverTool)
bonus_por_mes = data_filtrado.groupby('competencia')['bonus_numeric'].sum().reindex(ordem_meses)
source_p1 = ColumnDataSource(data=dict(
    competencia=ordem_meses,
    bonus=bonus_por_mes.values
))

p1 = bp.figure(x_range=ordem_meses, title="Bônus Total por Competência",
               width=600, height=350, toolbar_location="above",
               background_fill_color='white')
p1.vbar(x='competencia', top='bonus', width=0.7, color=azul_marinho, source=source_p1)
p1.xgrid.grid_line_color = None
p1.yaxis.axis_label = "Valor do Bônus (R$)"
p1.xaxis.major_label_orientation = 45

hover_p1 = HoverTool(tooltips=[
    ("Competência", "@competencia"),
    ("Bônus Total", "R$ @bonus{0,0.00}")
])
p1.add_tools(hover_p1)

# 3. Gráfico de linhas: Evolução mensal por revenda (com HoverTool)
p2 = bp.figure(x_range=ordem_meses, title="Evolução Mensal por Revenda",
               width=600, height=350, toolbar_location="above",
               background_fill_color='white')

for idx, rev in enumerate(revendas):
    serie = data_filtrado[data_filtrado['revenda'] == rev].groupby('competencia')['bonus_numeric'].sum().reindex(ordem_meses, fill_value=0)
    source_rev = ColumnDataSource(data=dict(
        competencia=ordem_meses,
        bonus=serie.values,
        revenda=[f'Revenda {rev}'] * len(ordem_meses)
    ))
    p2.line('competencia', 'bonus', line_width=2, color=Blues8[idx % len(Blues8)], 
            legend_label=str(rev), source=source_rev)
    p2.circle('competencia', 'bonus', size=6, color=Blues8[idx % len(Blues8)], 
              source=source_rev, alpha=0.6)

p2.xaxis.major_label_orientation = 45
p2.yaxis.axis_label = "Valor do Bônus (R$)"
p2.legend.location = "top_left"
p2.legend.label_text_color = azul_marinho

hover_p2 = HoverTool(tooltips=[
    ("Revenda", "@revenda"),
    ("Competência", "@competencia"),
    ("Bônus", "R$ @bonus{0,0.00}")
])
p2.add_tools(hover_p2)

# 4. Gráfico de pizza: Distribuição de tipos de bônus (com HoverTool)
from math import pi

tipos = ['valor_bonus', 'trade', 'rebate', 'comissao_vd']
nomes_tipos = ['Valor Bônus', 'Trade', 'Rebate', 'Comissão VD']
cores_tipos = [azul_marinho, cinza_escuro, cinza, '#5A5A72']
valores = [data_filtrado[t].astype(float).sum() for t in tipos]
total = sum(valores)
percentuais = [(v/total * 100) if total > 0 else 0 for v in valores]

angles = [v/total * 2 * pi if total > 0 else 0 for v in valores]
starts = [0] + list(np.cumsum(angles[:-1]))
ends = list(np.cumsum(angles))

source_p3 = ColumnDataSource(data=dict(
    start=starts,
    end=ends,
    color=cores_tipos,
    nome=nomes_tipos,
    valor=valores,
    percentual=percentuais
))

p3 = bp.figure(title="Distribuição dos Tipos de Bônus", width=400, height=400, 
               toolbar_location="above", background_fill_color='white', 
               x_range=(-1,1), y_range=(-1,1))

p3.wedge(x=0, y=0, radius=0.8, start_angle='start', end_angle='end',
         color='color', legend_field='nome', alpha=0.8, source=source_p3)

p3.axis.visible = False
p3.grid.visible = False
p3.legend.label_text_color = azul_marinho

hover_p3 = HoverTool(tooltips=[
    ("Tipo", "@nome"),
    ("Valor", "R$ @valor{0,0.00}"),
    ("Percentual", "@percentual{0.00}%")
])
p3.add_tools(hover_p3)

# 5. Gráfico de barras: Top Revendas por Bônus Total (com HoverTool)
bonus_por_revenda = data_filtrado.groupby('revenda')['bonus_numeric'].sum().sort_values(ascending=False)
top_revendas = bonus_por_revenda.head(9)

source_p4 = ColumnDataSource(data=dict(
    revenda=[str(r) for r in top_revendas.index],
    bonus=top_revendas.values
))

p4 = bp.figure(x_range=[str(r) for r in top_revendas.index], 
               title="Top Revendas por Bônus Total", 
               width=600, height=350, 
               toolbar_location="above",
               background_fill_color='white')
p4.vbar(x='revenda', top='bonus', width=0.7, color=azul_marinho, 
        alpha=0.8, source=source_p4)
p4.xaxis.axis_label = "Revenda"
p4.yaxis.axis_label = "Bônus Total (R$)"
p4.xgrid.grid_line_color = None

hover_p4 = HoverTool(tooltips=[
    ("Revenda", "@revenda"),
    ("Bônus Total", "R$ @bonus{0,0.00}")
])
p4.add_tools(hover_p4)

# Layout do dashboard
layout = column(
    select_competencia,
    kpi_total,
    row(p1, p3),
    row(p2, p4)
)

bp.show(layout)



NameError: name 'ordem_meses' is not defined